The Movie Database API – 
- Objective: Use this API to analyze the top 5 movies, by genre, starring any of the top 10 paid actresses or the top 10 paid actors in the US.​

# Import Dependencies

In [50]:
#import Dependencies
import json 
import requests
import pandas as pd
import numpy as np
from pprint import pprint
import os
import datetime as dt
import collections
from azure.storage.blob import BlobClient
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv


# THBD API Connection

In [51]:
# load api key variable
load_dotenv()

True

In [52]:
# Set Authentication

API_KEY = os.getenv("client_api_key")
base_url = "https://api.themoviedb.org/"
auth_url = "https://api.themoviedb.org/3/authentication"

headers = {	"accept":          "application/json", 
	         "Authorization": f'Bearer {API_KEY}'}
params = {'': ''}

In [53]:
# Endpoints

#Movie Lists Folder
top_rated_movie = '3/movie/top_rated'
page = 1
top_rated_movie_params = {'language': 'en-US', 'page': {page}, 'region': 'US'}

# Genre List
genre_list = '3/genre/movie/list'
genre_list_param = {'language' : 'en'}

# Actors
movie_id = 569094
actor_info = f'/3/movie/{movie_id}/credits'
actor_info_param = {'movie_id' : {movie_id}}


In [54]:
# Check Authentication & Response check
def response_check():
    try:
        response = requests.get(auth_url,headers=headers)
        response.raise_for_status()
        print(auth_url + ' {}'.format(response) )
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
response_check()

https://api.themoviedb.org/3/authentication <Response [200]>


In [55]:
# Check Endpoint & Response check
def response_endpoint_check(end_point, headers, params):
    try:
        response = requests.get(base_url + end_point,headers=headers, params=params)
        response.raise_for_status()
        print(base_url + end_point + ' {}'.format(response) )
    except requests.exceptions.HTTPError as err:
        print("Http Error:",err)
    
response_endpoint_check(end_point=top_rated_movie,headers=headers, params=top_rated_movie_params)
response_endpoint_check(end_point=genre_list,headers=headers, params=genre_list_param)
response_endpoint_check(end_point=actor_info,headers=headers, params=actor_info_param)

https://api.themoviedb.org/3/movie/top_rated <Response [200]>
https://api.themoviedb.org/3/genre/movie/list <Response [200]>
https://api.themoviedb.org//3/movie/569094/credits <Response [200]>


## End point Parameters update functions

In [56]:
# Update movie id parameter inside the Actor endpoint within a function
def get_actor_endpoint(movie_id_info):
    return f'/3/movie/{movie_id_info}/credits'

In [57]:
#Update movie id parameter within a function
def get_actor_param(movie_id_info):
    return {'movie_id' : {movie_id_info}}

In [58]:
#Update page id parameter within a function
def get_movie_endpoint_param(page_info):
    return {'language': 'en-US', 'page': {page_info}, 'region': 'US'}

#  Import CSV: Top 10 Paid US Actors

In [59]:
# Import the CSV file & Load to Dataframe
file = "top_paid_us_actors.csv"
#Create relative path to the .csv file. 
file_path = os.path.join("Resources",file) 


# Read data into dataframe(df) and show top 10 All time paid US actors.  
top_paid_actor = pd.read_csv(file_path,encoding="utf8", sep=',')
top_ten_paid_actors = pd.DataFrame(top_paid_actor)
top_ten_paid_actors

,full_name
0,Samuel L. Jackson
1,Robert Downey Jr.
2,Tom Hanks
3,Tom Cruise
4,Zoe Saldana
5,Chris Pratt
6,Chris Hemsworth
7,Bradley Cooper
8,Chris Evans
9,Harrison Ford


### Convert Json to Dataframe

In [60]:
#Get Json Endpoint and convert to dataframe
def convert_json_dataframe(endpoint , headers ,params=params):
    
    response = requests.get(base_url + endpoint, headers=headers, params=params)

    data = json.loads(response.text)

    format_data = json.dumps(data, indent=4)

    df = pd.read_json(format_data)

    return df


### JSON Nested Columns Extraction

In [61]:
#Get Nested Json Endpoint and convert to dataframe for overview of 1 page of records
def convert_json_nested_dataframe(endpoint , headers , record_paths, params=params, **argv):
    
    response = requests.get(base_url + endpoint, headers=headers, params=params)

    data = json.loads(response.text)
    
    for s in argv.values():
        f = pd.json_normalize(data, record_path=[record_paths],
                            meta=
                                        [s],
                                errors='ignore', record_prefix=' ')
    return f


### Find Duplicate Records

In [62]:
def find_duplicate_records(dataframe,col):
    return dataframe[dataframe.duplicated([col])]
    

## Table: Top Rated US Movies

In [63]:
# First look at Top Rated Movies Table
top_rated_movie_df = convert_json_dataframe(endpoint = top_rated_movie, headers=headers,
                                            params=top_rated_movie_params)
top_rated_movie_df.head(1)

,page,results,total_pages,total_results
0,1,"{'adult': False, 'backdrop_path': '/nGxUxi3PfX...",512,10225


### Top Rated Movies: Nested Columns Extracted

In [64]:
# The results column has been extracted to showcase nested results of the Top Rated Movies
nested_columns = ['id']
record_paths = 'results'
top_rated_movie_details_df = convert_json_nested_dataframe(endpoint = top_rated_movie, headers=headers, 
                                                           record_paths = record_paths, argv = nested_columns, 
                                                           params=top_rated_movie_params )
top_rated_movie_details_df.head(5)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,id
0,False,/nGxUxi3PfXDRm7Vg95VBNgNM8yc.jpg,"[28, 12, 16, 878]",569094,en,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2859.047,/8Vt6mWEReuy4Of61Lnj5Xj704m8.jpg,2023-06-02,Spider-Man: Across the Spider-Verse,False,8.8,1160,NaN
1,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",110.377,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,The Godfather,False,8.7,18076,NaN
2,False,/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,69.056,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,1994-09-23,The Shawshank Redemption,False,8.7,23935,NaN
3,False,/oo4PVK6AyLZN49BokxDFGyclN86.jpg,"[18, 80]",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,54.990,/bMadFzhjy9T7R8J48QGq1ngWNAK.jpg,1974-12-20,The Godfather Part II,False,8.6,10913,NaN
4,False,/vI3aUGTuRRdM7J78KIdW98LdxE5.jpg,"[35, 18, 10749]",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",26.908,/ktejodbcdCPXbMMdnpI9BUxW6O8.jpg,1995-10-20,Dilwale Dulhania Le Jayenge,False,8.6,4147,NaN


### Extract 30 Pages of Top Rated US Movie Records

In [65]:
movie_rows = []
nested_columnss = ['title']
count = 1
pages = 30
record_path = 'results'

while count < pages:
    movie_info_df = convert_json_nested_dataframe(endpoint = top_rated_movie , headers=headers, 
                                                  record_paths = record_path, argv = nested_columnss, 
                                                  params=get_movie_endpoint_param(count))
    df = pd.DataFrame(movie_info_df)
    df.columns = df.columns.str.replace(' ','')
    movie_rows.append(df)
    count += 1



### Transform Top Rated US Movies Table

In [66]:
#Choose needed Columns        
extracted_movie_records_df = pd.concat(movie_rows, axis=0)
extracted_movie_records_df = extracted_movie_records_df.iloc[:, [2,3,9,10,12]]
extracted_movie_records_df



,genre_ids,id,release_date,title,vote_average
0,"[28, 12, 16, 878]",569094,2023-06-02,Spider-Man: Across the Spider-Verse,8.8
1,"[18, 80]",238,1972-03-14,The Godfather,8.7
2,"[18, 80]",278,1994-09-23,The Shawshank Redemption,8.7
3,"[18, 80]",240,1974-12-20,The Godfather Part II,8.6
4,"[35, 18, 10749]",19404,1995-10-20,Dilwale Dulhania Le Jayenge,8.6
...,...,...,...,...,...
15,"[14, 18, 9648]",5961,1983-06-17,Fanny and Alexander,7.8
16,"[16, 35, 18, 10751, 10749]",393559,2017-02-24,My Life as a Zucchini,7.8
17,"[18, 10749]",142,2005-12-09,Brokeback Mountain,7.8
18,"[80, 35]",107,2001-01-19,Snatch,7.8


### Extract first value of each nested genre id list

In [67]:
extracted_movie_records_df['genre_id'] = extracted_movie_records_df['genre_ids'].apply(lambda x:x[0])
extracted_movie_records_df

,genre_ids,id,release_date,title,vote_average,genre_id
0,"[28, 12, 16, 878]",569094,2023-06-02,Spider-Man: Across the Spider-Verse,8.8,28
1,"[18, 80]",238,1972-03-14,The Godfather,8.7,18
2,"[18, 80]",278,1994-09-23,The Shawshank Redemption,8.7,18
3,"[18, 80]",240,1974-12-20,The Godfather Part II,8.6,18
4,"[35, 18, 10749]",19404,1995-10-20,Dilwale Dulhania Le Jayenge,8.6,35
...,...,...,...,...,...,...
15,"[14, 18, 9648]",5961,1983-06-17,Fanny and Alexander,7.8,14
16,"[16, 35, 18, 10751, 10749]",393559,2017-02-24,My Life as a Zucchini,7.8,16
17,"[18, 10749]",142,2005-12-09,Brokeback Mountain,7.8,18
18,"[80, 35]",107,2001-01-19,Snatch,7.8,80


### Test that Extracted Movie records match up with unnested genre id's

In [68]:
#Notice that genres_id first value is the same as genre_id value
extracted_movie_records_df.query(' `id` == 779047')

,genre_ids,id,release_date,title,vote_average,genre_id
18,"[16, 18, 10751]",779047,2021-03-05,Us Again,7.9,16


## Transformed Table

In [69]:
# Final Movie Details Table
the_movie_info_df = pd.DataFrame(extracted_movie_records_df)
the_movie_info_df = extracted_movie_records_df.drop('genre_ids', axis=1)
the_movie_info_df

,id,release_date,title,vote_average,genre_id
0,569094,2023-06-02,Spider-Man: Across the Spider-Verse,8.8,28
1,238,1972-03-14,The Godfather,8.7,18
2,278,1994-09-23,The Shawshank Redemption,8.7,18
3,240,1974-12-20,The Godfather Part II,8.6,18
4,19404,1995-10-20,Dilwale Dulhania Le Jayenge,8.6,35
...,...,...,...,...,...
15,5961,1983-06-17,Fanny and Alexander,7.8,14
16,393559,2017-02-24,My Life as a Zucchini,7.8,16
17,142,2005-12-09,Brokeback Mountain,7.8,18
18,107,2001-01-19,Snatch,7.8,80


#### Check for duplicated records in Top Rated US Movies


In [70]:
# Check for duplicates
find_duplicate_records(dataframe = the_movie_info_df,col= "id")


,id,release_date,title,vote_average,genre_id


### Drop duplicate records

In [71]:
# keep first duplicate row
the_movie_info_df = the_movie_info_df.drop_duplicates()

# Check for duplicates
find_duplicate_records(dataframe = the_movie_info_df,col= "id")

,id,release_date,title,vote_average,genre_id


### Movie ID and Genre ID Dataframe

In [72]:
#Create new data frame that has only genre and movie Id's to join with another table
genre_movie_id_df = pd.DataFrame(the_movie_info_df)
genre_movie_id_df = genre_movie_id_df.drop(['release_date','title','vote_average'], axis=1)
genre_movie_id_df


,id,genre_id
0,569094,28
1,238,18
2,278,18
3,240,18
4,19404,35
...,...,...
15,5961,14
16,393559,16
17,142,18
18,107,80


## Table: Genre

In [73]:
# First look at Genre Table
genre_list_nested_df = convert_json_dataframe(endpoint = genre_list , headers=headers , params=genre_list_param)
genre_list_nested_df.head(1)

,genres
0,"{'id': 28, 'name': 'Action'}"


### Genres: Nested Columns Extracted

In [74]:
nested_columns = ['id']
record_paths = 'genres'

genre_list_df = convert_json_nested_dataframe(endpoint = genre_list , headers=headers , record_paths = record_paths, argv = nested_columns, params=genre_list_param )
genre_list_df = genre_list_df.drop('id', axis=1)
genre_list_df.columns = genre_list_df.columns.str.replace(' ','')
genre_list_df.rename(columns={'id': 'genres_id','name': 'genre_name'}, inplace=True)
genre_list_df

,genres_id,genre_name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


#### Check for duplicated records in Genre Table

In [75]:
# Check for duplicates in genre table
find_duplicate_records(dataframe = genre_list_df,col= "genres_id")

,genres_id,genre_name


## Table: Actor Details

In [76]:
# Extract json nested list to dataframe
actor_rows = []
nested_columnss = ['genre']

for i in genre_movie_id_df['id']:
    actor_info_df = convert_json_nested_dataframe(endpoint = get_actor_endpoint(i) , headers=headers , record_paths = 'cast', argv = nested_columnss, params=get_actor_param(i) )
    actor_df = pd.DataFrame(actor_info_df).reset_index()
    actor_df.columns = actor_df.columns.str.replace(' ','')
    actor_df['movie_id'] = i
    actor_df = actor_df.head(1)
    actor_rows.append(actor_df)
       
extract_df = pd.concat(actor_rows, axis=0)
extract_df = extract_df.astype({'id':'int'})
extract_df = extract_df.astype({'gender':'int'})
extract_df.rename(columns={'id': 'actor_id'}, inplace=True) 

# Drop unneeded columns
actors_details_df = extract_df.drop(['index','adult','original_name', 'popularity', 'profile_path', 'cast_id', 'credit_id', 'order','genre'], axis=1)

actors_details_df

,gender,actor_id,known_for_department,name,character,movie_id
0,2,587506,Acting,Shameik Moore,Miles Morales / Spider-Man (voice),569094
0,2,1158,Acting,Al Pacino,Michael Corleone,238
0,2,504,Acting,Tim Robbins,Andy Dufresne,278
0,2,1158,Acting,Al Pacino,Don Michael Corleone,240
0,2,35742,Acting,Shah Rukh Khan,Raj Malhotra,19404
...,...,...,...,...,...,...
0,1,46865,Acting,Pernilla Allwin,Fanny Ekdahl,5961
0,0,1615534,Acting,Gaspard Schlatter,Courgette (voice),393559
0,2,1810,Acting,Heath Ledger,Ennis Del Mar,142
0,2,976,Acting,Jason Statham,Turkish,107


#### Check for duplicated records in Actor Details Table

In [77]:
find_duplicate_records(dataframe = actors_details_df,col= "movie_id")

,gender,actor_id,known_for_department,name,character,movie_id


In [78]:
join_df=pd.merge(genre_movie_id_df,actors_details_df, left_on=['id'],right_on=['movie_id'], how='left')
join_df

,id,genre_id,gender,actor_id,known_for_department,name,character,movie_id
0,569094,28,2.0,587506.0,Acting,Shameik Moore,Miles Morales / Spider-Man (voice),569094.0
1,238,18,2.0,1158.0,Acting,Al Pacino,Michael Corleone,238.0
2,278,18,2.0,504.0,Acting,Tim Robbins,Andy Dufresne,278.0
3,240,18,2.0,1158.0,Acting,Al Pacino,Don Michael Corleone,240.0
4,19404,35,2.0,35742.0,Acting,Shah Rukh Khan,Raj Malhotra,19404.0
...,...,...,...,...,...,...,...,...
575,5961,14,1.0,46865.0,Acting,Pernilla Allwin,Fanny Ekdahl,5961.0
576,393559,16,0.0,1615534.0,Acting,Gaspard Schlatter,Courgette (voice),393559.0
577,142,18,2.0,1810.0,Acting,Heath Ledger,Ennis Del Mar,142.0
578,107,80,2.0,976.0,Acting,Jason Statham,Turkish,107.0


##### These particular records did not have known_for_department: Acting information when further researched

In [79]:
join_df[join_df.isna().any(axis=1)]

,id,genre_id,gender,actor_id,known_for_department,name,character,movie_id
151,399106,10751,NaN,NaN,NaN,NaN,NaN,NaN
184,831827,16,NaN,NaN,NaN,NaN,NaN,NaN
254,574074,16,NaN,NaN,NaN,NaN,NaN,NaN
498,779047,16,NaN,NaN,NaN,NaN,NaN,NaN


### Actor Table

In [80]:
# Drop Nan records and transform datatypes
transform_joined_table=join_df.dropna().reset_index(drop=True)

transform_joined_table = transform_joined_table.astype({'gender':'int'})
transform_joined_table = transform_joined_table.astype({'actor_id':'int'})
transform_joined_table = transform_joined_table.astype({'movie_id':'int'})
actor_known_details_df = transform_joined_table
actor_known_details_df

,id,genre_id,gender,actor_id,known_for_department,name,character,movie_id
0,569094,28,2,587506,Acting,Shameik Moore,Miles Morales / Spider-Man (voice),569094
1,238,18,2,1158,Acting,Al Pacino,Michael Corleone,238
2,278,18,2,504,Acting,Tim Robbins,Andy Dufresne,278
3,240,18,2,1158,Acting,Al Pacino,Don Michael Corleone,240
4,19404,35,2,35742,Acting,Shah Rukh Khan,Raj Malhotra,19404
...,...,...,...,...,...,...,...,...
571,5961,14,1,46865,Acting,Pernilla Allwin,Fanny Ekdahl,5961
572,393559,16,0,1615534,Acting,Gaspard Schlatter,Courgette (voice),393559
573,142,18,2,1810,Acting,Heath Ledger,Ennis Del Mar,142
574,107,80,2,976,Acting,Jason Statham,Turkish,107


## Merge Actor Details and Movie Details Tables

In [81]:
movie_genre_actor_join_df=pd.merge(the_movie_info_df,actor_known_details_df, on=['id'], how='inner')
movie_genre_actor_join_df = movie_genre_actor_join_df.drop(['movie_id','genre_id_x'], axis=1)
movie_genre_actor_join_df.rename(columns={'genre_id_y': 'genre_id', 'name' : 'actor_name'}, inplace=True) 
movie_genre_actor_join_df

,id,release_date,title,vote_average,genre_id,gender,actor_id,known_for_department,actor_name,character
0,569094,2023-06-02,Spider-Man: Across the Spider-Verse,8.8,28,2,587506,Acting,Shameik Moore,Miles Morales / Spider-Man (voice)
1,238,1972-03-14,The Godfather,8.7,18,2,1158,Acting,Al Pacino,Michael Corleone
2,278,1994-09-23,The Shawshank Redemption,8.7,18,2,504,Acting,Tim Robbins,Andy Dufresne
3,240,1974-12-20,The Godfather Part II,8.6,18,2,1158,Acting,Al Pacino,Don Michael Corleone
4,19404,1995-10-20,Dilwale Dulhania Le Jayenge,8.6,35,2,35742,Acting,Shah Rukh Khan,Raj Malhotra
...,...,...,...,...,...,...,...,...,...,...
571,5961,1983-06-17,Fanny and Alexander,7.8,14,1,46865,Acting,Pernilla Allwin,Fanny Ekdahl
572,393559,2017-02-24,My Life as a Zucchini,7.8,16,0,1615534,Acting,Gaspard Schlatter,Courgette (voice)
573,142,2005-12-09,Brokeback Mountain,7.8,18,2,1810,Acting,Heath Ledger,Ennis Del Mar
574,107,2001-01-19,Snatch,7.8,80,2,976,Acting,Jason Statham,Turkish


## Top paid US Actors in Top Rated Movies

In [82]:
#Check if any top paid actors are in the joined dataframe
top_ten_paid_actors['in_top_movie_list'] =  top_ten_paid_actors['full_name'].isin(movie_genre_actor_join_df['actor_name'])
top_ten_paid_actors

,full_name,in_top_movie_list
0,Samuel L. Jackson,False
1,Robert Downey Jr.,True
2,Tom Hanks,True
3,Tom Cruise,True
4,Zoe Saldana,False
5,Chris Pratt,True
6,Chris Hemsworth,False
7,Bradley Cooper,False
8,Chris Evans,True
9,Harrison Ford,True


### Find all records that have paid actors in the top movies

In [83]:
#Find all records that have paid actors in the top movies
confirmed_movie_paid_actor_df=pd.merge(movie_genre_actor_join_df,top_ten_paid_actors, left_on=['actor_name'],right_on=['full_name'], how='right')
confirmed_movie_paid_actor_df


,id,release_date,title,vote_average,genre_id,gender,actor_id,known_for_department,actor_name,character,full_name,in_top_movie_list
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Samuel L. Jackson,False
1,299534.0,2019-04-26,Avengers: Endgame,8.3,12.0,2.0,3223.0,Acting,Robert Downey Jr.,Tony Stark / Iron Man,Robert Downey Jr.,True
2,299536.0,2018-04-27,Avengers: Infinity War,8.3,12.0,2.0,3223.0,Acting,Robert Downey Jr.,Tony Stark / Iron Man,Robert Downey Jr.,True
3,497.0,1999-12-10,The Green Mile,8.5,14.0,2.0,31.0,Acting,Tom Hanks,Paul Edgecomb,Tom Hanks,True
4,13.0,2014-09-05,Forrest Gump,8.5,35.0,2.0,31.0,Acting,Tom Hanks,Forrest Gump,Tom Hanks,True
5,857.0,1998-07-24,Saving Private Ryan,8.2,18.0,2.0,31.0,Acting,Tom Hanks,Captain John H. Miller,Tom Hanks,True
6,862.0,1995-11-22,Toy Story,8.0,16.0,2.0,31.0,Acting,Tom Hanks,Woody (voice),Tom Hanks,True
7,522402.0,2021-11-04,Finch,8.0,878.0,2.0,31.0,Acting,Tom Hanks,Finch,Tom Hanks,True
8,937278.0,2022-12-30,A Man Called Otto,7.9,35.0,2.0,31.0,Acting,Tom Hanks,Otto Anderson,Tom Hanks,True
9,361743.0,2022-05-27,Top Gun: Maverick,8.3,28.0,2.0,500.0,Acting,Tom Cruise,Capt. Pete 'Maverick' Mitchell,Tom Cruise,True


### Drop records that are nan for top paid actors not in top rated movies

In [84]:
# Check for NAN values
confirmed_movie_paid_actor_df[confirmed_movie_paid_actor_df.isna().any(axis=1)]
confirmed_movie_paid_actor_df

#Drop those records
paid_actor_in_movie_df=confirmed_movie_paid_actor_df.dropna().reset_index(drop=True)
paid_actor_in_movie_df = paid_actor_in_movie_df.astype({'genre_id':'int'})
paid_actor_in_movie_df = paid_actor_in_movie_df.astype({'gender':'int'})
paid_actor_in_movie_df = paid_actor_in_movie_df.astype({'actor_id':'int'})
paid_actor_in_movie_df = paid_actor_in_movie_df.astype({'id':'int'})

# Final table with Top paid actors in 500 movies
paid_actor_in_movie_df

,id,release_date,title,vote_average,genre_id,gender,actor_id,known_for_department,actor_name,character,full_name,in_top_movie_list
0,299534,2019-04-26,Avengers: Endgame,8.3,12,2,3223,Acting,Robert Downey Jr.,Tony Stark / Iron Man,Robert Downey Jr.,True
1,299536,2018-04-27,Avengers: Infinity War,8.3,12,2,3223,Acting,Robert Downey Jr.,Tony Stark / Iron Man,Robert Downey Jr.,True
2,497,1999-12-10,The Green Mile,8.5,14,2,31,Acting,Tom Hanks,Paul Edgecomb,Tom Hanks,True
3,13,2014-09-05,Forrest Gump,8.5,35,2,31,Acting,Tom Hanks,Forrest Gump,Tom Hanks,True
4,857,1998-07-24,Saving Private Ryan,8.2,18,2,31,Acting,Tom Hanks,Captain John H. Miller,Tom Hanks,True
5,862,1995-11-22,Toy Story,8.0,16,2,31,Acting,Tom Hanks,Woody (voice),Tom Hanks,True
6,522402,2021-11-04,Finch,8.0,878,2,31,Acting,Tom Hanks,Finch,Tom Hanks,True
7,937278,2022-12-30,A Man Called Otto,7.9,35,2,31,Acting,Tom Hanks,Otto Anderson,Tom Hanks,True
8,361743,2022-05-27,Top Gun: Maverick,8.3,28,2,500,Acting,Tom Cruise,Capt. Pete 'Maverick' Mitchell,Tom Cruise,True
9,447365,2023-05-05,Guardians of the Galaxy Vol. 3,8.1,878,2,73457,Acting,Chris Pratt,Peter Quill / Star-Lord,Chris Pratt,True


## Merge Genre table with Paid Actors in Top Movies Table

In [85]:
paid_actor_in_movie_df2 = pd.merge(paid_actor_in_movie_df,genre_list_df, left_on=['genre_id'],right_on=['genres_id'], how='left')
paid_actor_in_movie_df2 = paid_actor_in_movie_df2.sort_values(by=['vote_average'], ascending=False)
paid_actor_in_movie_df2 = paid_actor_in_movie_df2.drop(['full_name','in_top_movie_list','genres_id','known_for_department'], axis=1)
paid_actor_in_movie_df2

,id,release_date,title,vote_average,genre_id,gender,actor_id,actor_name,character,genre_name
2,497,1999-12-10,The Green Mile,8.5,14,2,31,Tom Hanks,Paul Edgecomb,Fantasy
3,13,2014-09-05,Forrest Gump,8.5,35,2,31,Tom Hanks,Forrest Gump,Comedy
0,299534,2019-04-26,Avengers: Endgame,8.3,12,2,3223,Robert Downey Jr.,Tony Stark / Iron Man,Adventure
1,299536,2018-04-27,Avengers: Infinity War,8.3,12,2,3223,Robert Downey Jr.,Tony Stark / Iron Man,Adventure
8,361743,2022-05-27,Top Gun: Maverick,8.3,28,2,500,Tom Cruise,Capt. Pete 'Maverick' Mitchell,Action
4,857,1998-07-24,Saving Private Ryan,8.2,18,2,31,Tom Hanks,Captain John H. Miller,Drama
9,447365,2023-05-05,Guardians of the Galaxy Vol. 3,8.1,878,2,73457,Chris Pratt,Peter Quill / Star-Lord,Science Fiction
12,400928,2017-04-12,Gifted,8.1,18,2,16828,Chris Evans,Frank Adler,Drama
5,862,1995-11-22,Toy Story,8.0,16,2,31,Tom Hanks,Woody (voice),Animation
6,522402,2021-11-04,Finch,8.0,878,2,31,Tom Hanks,Finch,Science Fiction


# Analysis

### Top 5 Movies by genre who has a Top paid actor starring in the movie

In [86]:
#drop unneeded columns
paid_actor_in_movie_df3 = paid_actor_in_movie_df2.drop(['genre_id','gender','actor_id'],axis=1)
top_five_movie_paid_actor_in = paid_actor_in_movie_df3.head(5).reset_index().drop(columns='index')

# Add date in which results was captured
top_five_movie_paid_actor_in['analysis_date'] = dt.datetime.today().strftime('%Y-%m-%d')
top_five_movie_paid_actor_in

,id,release_date,title,vote_average,actor_name,character,genre_name,analysis_date
0,497,1999-12-10,The Green Mile,8.5,Tom Hanks,Paul Edgecomb,Fantasy,2023-06-13
1,13,2014-09-05,Forrest Gump,8.5,Tom Hanks,Forrest Gump,Comedy,2023-06-13
2,299534,2019-04-26,Avengers: Endgame,8.3,Robert Downey Jr.,Tony Stark / Iron Man,Adventure,2023-06-13
3,299536,2018-04-27,Avengers: Infinity War,8.3,Robert Downey Jr.,Tony Stark / Iron Man,Adventure,2023-06-13
4,361743,2022-05-27,Top Gun: Maverick,8.3,Tom Cruise,Capt. Pete 'Maverick' Mitchell,Action,2023-06-13


### All Top rated Movies by Genre who has a Top paid actor starring in the movie

In [87]:
genre_series = paid_actor_in_movie_df2.groupby(['genre_name', 'actor_name','vote_average']).apply(lambda x: x['title'].reset_index(drop=True))
genre_df = pd.DataFrame(genre_series).reset_index().drop(columns='level_3').sort_values(by=['genre_name','vote_average'], ascending=False)

genre_df = genre_df.reset_index().drop(columns='index')
genre_df

,genre_name,actor_name,vote_average,title
0,Science Fiction,Chris Pratt,8.1,Guardians of the Galaxy Vol. 3
1,Science Fiction,Tom Hanks,8.0,Finch
2,Science Fiction,Harrison Ford,7.9,Blade Runner
3,Fantasy,Tom Hanks,8.5,The Green Mile
4,Drama,Tom Hanks,8.2,Saving Private Ryan
5,Drama,Chris Evans,8.1,Gifted
6,Comedy,Tom Hanks,8.5,Forrest Gump
7,Comedy,Tom Hanks,7.9,A Man Called Otto
8,Animation,Tom Hanks,8.0,Toy Story
9,Animation,Chris Pratt,7.8,The Super Mario Bros. Movie


### Actor with the most starring roles in top rated movies

In [88]:
most_starring_actor = genre_df.groupby('actor_name').count().reset_index().sort_values(by=['title','vote_average'], ascending=False)
most_starring_actor = most_starring_actor.drop(['genre_name','vote_average'],axis=1).reset_index().drop(columns='index')
most_starring_actor

,actor_name,title
0,Tom Hanks,6
1,Chris Pratt,3
2,Harrison Ford,3
3,Robert Downey Jr.,2
4,Chris Evans,1
5,Tom Cruise,1


### Genre with the most starring roles in top rated movies

In [89]:
most_genres_with_starring_actor = genre_df.groupby('genre_name').count().reset_index().sort_values(by=['actor_name'], ascending=False)
most_genres_with_starring_actor = most_genres_with_starring_actor.drop(['title','vote_average'],axis=1).reset_index().drop(columns='index')
most_genres_with_starring_actor

,genre_name,actor_name
0,Adventure,4
1,Science Fiction,3
2,Action,2
3,Animation,2
4,Comedy,2
5,Drama,2
6,Fantasy,1


# Export Dataframe to CSV

In [90]:
def update_file_name(file):
    return os.path.join("Resources",file)

In [91]:
result_file = "top_five_movies_with_top_paid_actors.csv"
today_date = dt.datetime.today().strftime("%Y-%m-%d")
#print(today_date)
updated_result_file = f"top_five_movies_with_top_paid_actors{today_date}.csv"                                                
file_paths = update_file_name(updated_result_file)

In [92]:
#send to csv
result_file = top_five_movie_paid_actor_in.to_csv(update_file_name(updated_result_file),
                                                                        encoding='utf-8', index=False)
result_file


# Export file to Azure storage

In [93]:
STORAGE_ACCOUNT_KEY = os.getenv("storage_account_key")
STORAGE_ACCOUNT_NAME = os.getenv("storage_account_name")
connection_string = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"
container_name = "python-capstone-nage"


def uploadToBlobStorage(file_path, file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    
    with open(file_path,"rb") as data:
        blob_client.upload_blob(data) #include overwrite=True, if you want to overwrite file
        return print(f"Uploaded {file_name}")


In [94]:
#upload to storage
uploadToBlobStorage(file_path = file_paths, file_name = updated_result_file)

Uploaded top_five_movies_with_top_paid_actors2023-06-13.csv


## List Blobs in the container

In [95]:
service = BlobServiceClient.from_connection_string(conn_str=connection_string)
container_client = service.get_container_client(container_name)
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')

Resources

Resources/top_paid_actors.csv

top_five_movies_with_top_paid_actors.csv

top_five_movies_with_top_paid_actors2023-06-13.csv

